In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import imageio.v3 as imageio

from tqdm.notebook import tqdm

import librosa
import cv2
import pickle
import lzma

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for entry in os.listdir('/kaggle/input/birdclef-2024/train_audio'):
print(os.listdir('/kaggle/input/birdclef-2024/train_audio'))
print(len(os.listdir('/kaggle/input/birdclef-2024/train_audio')))

train_species = os.listdir('/kaggle/input/birdclef-2024/train_audio')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


['redspu1', 'zitcis1', 'marsan', 'rewlap1', 'grenig1', 'insbab1', 'commyn', 'labcro1', 'yebbul3', 'whtkin2', 'brasta1', 'brnhao1', 'bkcbul1', 'grywag', 'graher1', 'brodro1', 'purher1', 'bkskit1', 'eurbla2', 'grtdro1', 'comtai1', 'paisto1', 'brcful1', 'grehor1', 'whcbar1', 'blakit1', 'integr', 'gryfra', 'grnsan', 'whbsho3', 'sqtbul1', 'greegr', 'wemhar1', 'comsan', 'rewbul', 'cohcuc1', 'heswoo1', 'gybpri1', 'gargan', 'darter2', 'rerswa1', 'grnwar1', 'isbduc1', 'brwjac1', 'mawthr1', 'eaywag1', 'pursun3', 'lblwar1', 'kerlau2', 'junmyn1', 'rutfly6', 'oripip1', 'houspa', 'lobsun2', 'wynlau1', 'eucdov', 'compea', 'spodov', 'nutman', 'revbul', 'aspswi1', 'blaeag1', 'whiter2', 'litspi1', 'litswi1', 'rufwoo2', 'rossta2', 'barfly1', 'ruftre2', 'inpher1', 'maghor2', 'litgre1', 'grynig2', 'lirplo', 'rorpar', 'gloibi', 'comkin1', 'orihob2', 'sohmyn1', 'yebbab1', 'plapri1', 'stbkin1', 'blhori1', 'comros', 'ashdro1', 'dafbab1', 'ingori1', 'rocpig', 'jerbus2', 'inbrob1', 'houcro1', 'asbfly', 'whbwat1'

In [2]:
# need a small piece of code to convert the audio directory to [file_path, metadata for the file path, label]
# convert audio directory to [file_path, label], then join metadata with file_path
import os
dataset = []

# Path to the input directory
input_dir = '/kaggle/input/birdclef-2024/train_audio'

# Iterate through each subdirectory
for subdir_name in os.listdir(input_dir):
    subdir_path = os.path.join(input_dir, subdir_name)
    
    # Check if it's a directory
    if os.path.isdir(subdir_path):
        # Iterate through files in the subdirectory
        for file_name in os.listdir(subdir_path):
            full_file_path = os.path.join(subdir_path, file_name)
            file_path = os.path.join(subdir_name, file_name)
            # Append [subdirectory_name, full_file_path] to the dataset list
            dataset.append([subdir_name, file_path, full_file_path])

# Print the dataset
# for data in dataset:
#     print(data)

print(dataset[1:10])

# Convert the list of lists to a pandas DataFrame
dataset_df = pd.DataFrame(dataset, columns=['subdir_name', 'file_name', 'full_file_path'])

print(type(dataset_df))
print(dataset_df[dataset_df['subdir_name'] == 'brfowl1'])

[['redspu1', 'redspu1/XC37584.ogg', '/kaggle/input/birdclef-2024/train_audio/redspu1/XC37584.ogg'], ['redspu1', 'redspu1/XC283501.ogg', '/kaggle/input/birdclef-2024/train_audio/redspu1/XC283501.ogg'], ['redspu1', 'redspu1/XC469479.ogg', '/kaggle/input/birdclef-2024/train_audio/redspu1/XC469479.ogg'], ['redspu1', 'redspu1/XC471046.ogg', '/kaggle/input/birdclef-2024/train_audio/redspu1/XC471046.ogg'], ['redspu1', 'redspu1/XC760790.ogg', '/kaggle/input/birdclef-2024/train_audio/redspu1/XC760790.ogg'], ['redspu1', 'redspu1/XC623939.ogg', '/kaggle/input/birdclef-2024/train_audio/redspu1/XC623939.ogg'], ['redspu1', 'redspu1/XC356035.ogg', '/kaggle/input/birdclef-2024/train_audio/redspu1/XC356035.ogg'], ['redspu1', 'redspu1/XC634745.ogg', '/kaggle/input/birdclef-2024/train_audio/redspu1/XC634745.ogg'], ['redspu1', 'redspu1/XC775566.ogg', '/kaggle/input/birdclef-2024/train_audio/redspu1/XC775566.ogg']]
<class 'pandas.core.frame.DataFrame'>
      subdir_name             file_name  \
17636     b

In [3]:
# just for validation purposes to see if we have handled the file paths correctly
test_input_dir = '/kaggle/input/birdclef-2024/train_audio/brfowl1'
print(len(os.listdir(test_input_dir)))
for subdir_name in os.listdir(test_input_dir):
    print(subdir_name)

17
XC361783.ogg
XC138097.ogg
XC446902.ogg
XC138098.ogg
XC826279.ogg
XC824089.ogg
XC522123.ogg
XC824091.ogg
XC124214.ogg
XC593653.ogg
XC124221.ogg
XC304563.ogg
XC824093.ogg
XC824092.ogg
XC542628.ogg
XC824090.ogg
XC124213.ogg


<h2>Notes about intial data investigation </h2>
1. 1. for taxonomy, species_group is going to be not always populated. the dataset seems to only populate the same entry for the group. Need to backfill all species group. Reported as are all NaN for the 182 species we concern. So probably don't need to handle it now.
1. 2. need to do a bit more investigation on taxonomy datasets, and merge it into the main dataset (merged_df)


In [4]:
train_metadata = pd.read_csv('/kaggle/input/birdclef-2024/train_metadata.csv')
print(len(train_metadata)) # 24459

unique_file_name = train_metadata['filename'].unique()
print(len(unique_file_name)) # 24459

# Merge the two datasets on the 'filename' column
merged_df = pd.merge(dataset_df, train_metadata, left_on='file_name', right_on='filename', how='inner')

# Print the merged DataFrame
merged_df.head()

24459
24459


,subdir_name,file_name,full_file_path,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename
0,redspu1,redspu1/XC312771.ogg,/kaggle/input/birdclef-2024/train_audio/redspu...,redspu1,[],"['alarm call', 'call', 'juvenile']",19.203200,72.919800,Galloperdix spadicea,Red Spurfowl,Yogesh Patel,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/312771,redspu1/XC312771.ogg
1,redspu1,redspu1/XC37584.ogg,/kaggle/input/birdclef-2024/train_audio/redspu...,redspu1,[],['song'],12.050317,76.130705,Galloperdix spadicea,Red Spurfowl,David Farrow,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/37584,redspu1/XC37584.ogg
2,redspu1,redspu1/XC283501.ogg,/kaggle/input/birdclef-2024/train_audio/redspu...,redspu1,[],['call'],10.408400,76.719100,Galloperdix spadicea,Red Spurfowl,Vivek Puliyeri,Creative Commons Attribution-NonCommercial-Sha...,3.0,https://www.xeno-canto.org/283501,redspu1/XC283501.ogg
3,redspu1,redspu1/XC469479.ogg,/kaggle/input/birdclef-2024/train_audio/redspu...,redspu1,[],['call'],11.275900,76.218800,Galloperdix spadicea,Red Spurfowl,Vivek Puliyeri,Creative Commons Attribution-NonCommercial-Sha...,3.0,https://www.xeno-canto.org/469479,redspu1/XC469479.ogg
4,redspu1,redspu1/XC471046.ogg,/kaggle/input/birdclef-2024/train_audio/redspu...,redspu1,[],"['call', 'song']",15.533400,73.828600,Galloperdix spadicea,Red Spurfowl,Tejaswini Limaye,Creative Commons Attribution-NonCommercial-Sha...,3.0,https://www.xeno-canto.org/471046,redspu1/XC471046.ogg


In [5]:
# Code blocks for converting the sound to spectrograms (copied from other example notebooks)
class Config():
    # Horizontal melspectrogram resolution
    MELSPEC_H = 128
    # Competition Root Folder
    ROOT_FOLDER = '/kaggle/input/birdclef-2024'
    # Maximum decibel to clip audio to
    TOP_DB = 100
    # Minimum rating
    MIN_RATING = 3.0
    # Sample rate as provided in competition description
    SR = 32000
    
CONFIG = Config()

# Convert OOG audio files to melspectrogram encoded as PNG bytes
def ogg2melspectrogram(file_path):
    # Load the audio file
    y, sr = librosa.load(file_path)
    # Convert to mel spectrogram
    spec = librosa.feature.melspectrogram(
        y=y,
        sr=32e3, # sample rate
        n_fft= 2048, # number of samples in window 
        hop_length=512, # step size of window
        n_mels=CONFIG.MELSPEC_H, # horizontal resolution from fmin→fmax in log scale
        fmin=40, # minimum frequency
        fmax=15000, # maximum frequency
        power=2.0, # intensity^power for log scale
    )
    # Convert to Db
    spec = librosa.power_to_db(spec, ref=CONFIG.TOP_DB)
    # Normalize 0-min
    spec = spec - spec.min()
    # Normalize 0-255
    spec = (spec / spec.max() * 255).astype(np.uint8)
    # Convert to PNG bytes
    _, spec_png_uint8 = cv2.imencode('.png', spec)
    spec_png_bytes = bytes(spec_png_uint8)
    
    return spec_png_bytes

In [6]:
# use the above audio conversion method ogg2ogg2melspectrogram to convert the raw files into bytes. Try how long it takes
merged_df['mel_spectrogram'] = merged_df['full_file_path'].apply(ogg2melspectrogram)

In [7]:
# check the size of the dataframe got converted
df_memory_usage_bytes = merged_df.memory_usage(deep=True).sum()

# Convert bytes to megabytes (MB)
df_memory_usage_mb = df_memory_usage_bytes / (1024 * 1024)

# Convert bytes to gigabytes (GB)
df_memory_usage_gb = df_memory_usage_bytes / (1024 * 1024 * 1024)

print("DataFrame memory usage in MB:", df_memory_usage_mb)
print("DataFrame memory usage in GB:", df_memory_usage_gb)

DataFrame memory usage in MB: 3482.5441274642944
DataFrame memory usage in GB: 3.40092199947685


In [8]:
# persist the converted file in memory so we can continue the work tomorrow
merged_df.to_csv('merged_data.csv', index=False, mode='w')

In [9]:
taxonomy = pd.read_csv('/kaggle/input/birdclef-2024/eBird_Taxonomy_v2021.csv')
taxonomy.head()


,TAXON_ORDER,CATEGORY,SPECIES_CODE,PRIMARY_COM_NAME,SCI_NAME,ORDER1,FAMILY,SPECIES_GROUP,REPORT_AS
0,1,species,ostric2,Common Ostrich,Struthio camelus,Struthioniformes,Struthionidae (Ostriches),Ostriches,NaN
1,6,species,ostric3,Somali Ostrich,Struthio molybdophanes,Struthioniformes,Struthionidae (Ostriches),NaN,NaN
2,7,slash,y00934,Common/Somali Ostrich,Struthio camelus/molybdophanes,Struthioniformes,Struthionidae (Ostriches),NaN,NaN
3,8,species,grerhe1,Greater Rhea,Rhea americana,Rheiformes,Rheidae (Rheas),Rheas,NaN
4,14,species,lesrhe2,Lesser Rhea,Rhea pennata,Rheiformes,Rheidae (Rheas),NaN,NaN


In [10]:
train_taxonomy = taxonomy[taxonomy['SPECIES_CODE'].isin(train_species)]
species_group = taxonomy[taxonomy['REPORT_AS'].notna()]
species_group.head()

,TAXON_ORDER,CATEGORY,SPECIES_CODE,PRIMARY_COM_NAME,SCI_NAME,ORDER1,FAMILY,SPECIES_GROUP,REPORT_AS
5,15,issf,lesrhe4,Lesser Rhea (Puna),Rhea pennata tarapacensis/garleppi,Rheiformes,Rheidae (Rheas),NaN,lesrhe2
6,18,issf,lesrhe3,Lesser Rhea (Darwin's),Rhea pennata pennata,Rheiformes,Rheidae (Rheas),NaN,lesrhe2
9,21,issf,higtin2,Highland Tinamou (South American),Nothocercus bonapartei [bonapartei Group],Tinamiformes,Tinamidae (Tinamous),NaN,higtin1
10,26,issf,higtin3,Highland Tinamou (Costa Rican),Nothocercus bonapartei frantzii,Tinamiformes,Tinamidae (Tinamous),NaN,higtin1
23,73,issf,brntin1,Brown Tinamou (castaneus),Crypturellus obsoletus castaneus,Tinamiformes,Tinamidae (Tinamous),NaN,brotin1
